In [1]:
import datetime
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import mean_absolute_percentage_error,mean_absolute_error

In [2]:
work_path=r'D:\university\研究生\tourism_forecasting_competition\submit\code and supplementary materials'##work_path must be adjusted before running the code.
stack_path=r'D:\university\研究生\tourism_forecasting_competition\submit\code and supplementary materials\stacking_train' 
baseline_path=r'D:\university\研究生\tourism_forecasting_competition\submit\code and supplementary materials\baseline_forecast'
os.chdir(work_path)

In [3]:
country_name=["加拿大",'智利','墨西哥','台湾','香港','日本','韩国','澳门','马尔代夫','柬埔寨','印尼','新加坡','新西兰','美国','泰国','土耳其','澳大利亚','夏威夷','奥地利','捷克']

In [7]:
data=pd.read_excel('data.xlsx').iloc[:,1:]
data.columns=country_name
data.index=pd.date_range(start="1989-1", end="2023-02", freq="MS")

In [58]:
###use Lasso as the meta model of stacking
baseline_stacking_lasso=pd.DataFrame(index=pd.date_range(start="2023-01", end="2024-12", freq="MS"))
train_stacking_lasso=pd.DataFrame(index=pd.date_range(start="2018-01", end="2019-12", freq="MS"))
stacking_train=pd.DataFrame()
stacking_test=pd.DataFrame()
for i in range(20):
    
    os.chdir(stack_path)
    for file in os.listdir(stack_path):
        temp=pd.read_excel(file)
        stacking_train[file[:-9]]=temp.iloc[:,i]
    y_train=np.array(data.iloc[348:372,i]).reshape(-1,1) 
    trainer=Lasso(random_state=42,positive=True,fit_intercept=False)
    trainer.fit(np.array(stacking_train),y_train)
    train_prediction=trainer.predict(np.array(stacking_train))
    train_stacking_lasso[country_name[i]]=train_prediction

    os.chdir(baseline_path)
    for file in os.listdir(baseline_path):
        temp=pd.read_excel(file)
        stacking_test[file[:-9]]=temp.iloc[:,i]
    test_prediction=trainer.predict(np.array(stacking_test))
    baseline_stacking_lasso[country_name[i]]=test_prediction


g:\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.888e+07, tolerance: 1.016e+07
  model = cd_fast.enet_coordinate_descent(
g:\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.241e+06, tolerance: 2.204e+04
  model = cd_fast.enet_coordinate_descent(
g:\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.007e+10, tolerance: 3.946e+10
  model = cd_fast.enet_

In [61]:
###use Ridge as the meta model of stacking
baseline_stacking_ridge=pd.DataFrame(index=pd.date_range(start="2023-01", end="2024-12", freq="MS"))
train_stacking_ridge=pd.DataFrame(index=pd.date_range(start="2018-01", end="2019-12", freq="MS"))
stacking_train=pd.DataFrame()
stacking_test=pd.DataFrame()
for i in range(20):
    
    os.chdir(stack_path)
    for file in os.listdir(stack_path):
        temp=pd.read_excel(file)
        stacking_train[file[:-9]]=temp.iloc[:,i]
    y_train=np.array(data.iloc[348:372,i]).reshape(-1,1) 
    trainer=Ridge(random_state=42,positive=True,fit_intercept=False)
    trainer.fit(np.array(stacking_train),y_train)
    train_prediction=trainer.predict(np.array(stacking_train))
    train_stacking_ridge[country_name[i]]=train_prediction.reshape(-1,1)

    os.chdir(baseline_path)
    for file in os.listdir(baseline_path):
        temp=pd.read_excel(file)
        stacking_test[file[:-9]]=temp.iloc[:,i]
    test_prediction=trainer.predict(np.array(stacking_test))
    baseline_stacking_ridge[country_name[i]]=test_prediction

In [36]:
### simple average
baseline_average=pd.DataFrame(index=pd.date_range(start="2023-01", end="2024-12", freq="MS"))
train_average=pd.DataFrame(index=pd.date_range(start="2018-01", end="2019-12", freq="MS"))
stacking_train=pd.DataFrame()
stacking_test=pd.DataFrame()
drop_method=['holt','rwf','TDha','TDhe','etsstlA','etsstlB','etsstlC']
for i in range(20):
    os.chdir(stack_path)
    for file in os.listdir(stack_path):
        temp=pd.read_excel(file)
        if file[:-9] not in drop_method:
            stacking_train[file[:-9]]=temp.iloc[:,i]
    train_prediction=stacking_train.mean(axis=1)
    train_average[country_name[i]]=list(train_prediction)

    os.chdir(baseline_path)
    for file in os.listdir(baseline_path):
        temp=pd.read_excel(file)
        stacking_test[file[:-9]]=temp.iloc[:,i]
    test_prediction=stacking_test.mean(axis=1)
    baseline_average[country_name[i]]=list(test_prediction)

In [63]:
(np.abs(train_stacking_ridge-data.loc[datetime.datetime(2018,1,1):datetime.datetime(2019,12,1)]).mean()/np.abs(data.loc[datetime.datetime(2017,12,1):datetime.datetime(2019,12,1)].diff()).mean()).mean()

0.5922587075870022

In [64]:
np.abs((train_stacking_ridge-data.loc[datetime.datetime(2018,1,1):datetime.datetime(2019,12,1)])/data.loc[datetime.datetime(2018,1,1):datetime.datetime(2019,12,1)]).mean().mean()

0.10513961318326996

In [49]:
### error weighted average
baseline_weighted_average=pd.DataFrame(index=pd.date_range(start="2023-01", end="2024-12", freq="MS"))
train_weighted_average=pd.DataFrame(index=pd.date_range(start="2018-01", end="2019-12", freq="MS"))
stacking_train=pd.DataFrame()
stacking_test=pd.DataFrame()
error=pd.DataFrame(columns=['arima', 'arimastlA', 'arimastlB', 'arimastlC', 'ets','hw', 'mint', 'nnetar', 'snaive',
       'tbats', 'TDfa', 'TDfe', 'wls'])
drop_method=[]
# drop_method=['holt','rwf','TDha','TDhe','etsstlA','etsstlB','etsstlC']
for i in range(20):
    os.chdir(stack_path)
    for file in os.listdir(stack_path):
        print(file)
        temp=pd.read_excel(file)
        if file[:-9] not in drop_method:
            stacking_train[file[:-9]]=temp.iloc[:,i]
    y_train=np.array(data.iloc[348:372,i]).reshape(-1,1)
    dic=dict()
    for method in stacking_train.columns:
        naive_error=np.mean(np.abs(np.diff(y_train.reshape(-1))))
        prediction_error=mean_absolute_error(np.array(stacking_train[method]).reshape(-1,1),y_train)
        dic[method]=prediction_error/naive_error
        # dic[method]=mean_absolute_percentage_error(np.array(stacking_train[method]).reshape(-1,1),y_train)
    error=error.append(dic,ignore_index=True)
error_inverse=1/error.mean(axis=0)
weight=np.array((error_inverse/error_inverse.sum())).reshape(-1,1)
for i in range(20):
    os.chdir(stack_path)
    for file in os.listdir(stack_path):
        temp=pd.read_excel(file)
        if file[:-9] not in drop_method:
            stacking_train[file[:-9]]=temp.iloc[:,i]
    train_prediction=np.array(stacking_train)@weight
    train_weighted_average[country_name[i]]=train_prediction

    os.chdir(baseline_path)
    for file in os.listdir(baseline_path):
        temp=pd.read_excel(file)
        if file[:-9] not in drop_method:
            stacking_test[file[:-9]]=temp.iloc[:,i]
    test_prediction=np.array(stacking_test)@weight
    baseline_weighted_average[country_name[i]]=test_prediction

arima2018.xlsx
arimastlA2018.xlsx
arimastlB2018.xlsx
arimastlC2018.xlsx
ets2018.xlsx
etsstlA2018.xlsx
etsstlB2018.xlsx
etsstlC2018.xlsx
holt2018.xlsx
hw2018.xlsx
mint2018.xlsx
nnetar2018.xlsx
rwf2018.xlsx
snaive2018.xlsx
tbats2018.xlsx
TDfa2018.xlsx
TDfe2018.xlsx
TDha2018.xlsx
TDhe2018.xlsx
wls2018.xlsx
arima2018.xlsx
arimastlA2018.xlsx
arimastlB2018.xlsx
arimastlC2018.xlsx
ets2018.xlsx
etsstlA2018.xlsx
etsstlB2018.xlsx
etsstlC2018.xlsx
holt2018.xlsx
hw2018.xlsx
mint2018.xlsx
nnetar2018.xlsx
rwf2018.xlsx
snaive2018.xlsx


C:\Users\rtz12\AppData\Local\Temp\ipykernel_15192\1896440418.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error=error.append(dic,ignore_index=True)


tbats2018.xlsx
TDfa2018.xlsx
TDfe2018.xlsx
TDha2018.xlsx
TDhe2018.xlsx
wls2018.xlsx
arima2018.xlsx
arimastlA2018.xlsx
arimastlB2018.xlsx
arimastlC2018.xlsx
ets2018.xlsx
etsstlA2018.xlsx
etsstlB2018.xlsx
etsstlC2018.xlsx
holt2018.xlsx
hw2018.xlsx
mint2018.xlsx
nnetar2018.xlsx


C:\Users\rtz12\AppData\Local\Temp\ipykernel_15192\1896440418.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error=error.append(dic,ignore_index=True)


rwf2018.xlsx
snaive2018.xlsx
tbats2018.xlsx
TDfa2018.xlsx
TDfe2018.xlsx
TDha2018.xlsx
TDhe2018.xlsx
wls2018.xlsx
arima2018.xlsx
arimastlA2018.xlsx
arimastlB2018.xlsx
arimastlC2018.xlsx
ets2018.xlsx
etsstlA2018.xlsx
etsstlB2018.xlsx
etsstlC2018.xlsx
holt2018.xlsx


...

C:\Users\rtz12\AppData\Local\Temp\ipykernel_15192\1896440418.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error=error.append(dic,ignore_index=True)


TDfe2018.xlsx
TDha2018.xlsx
TDhe2018.xlsx
wls2018.xlsx
arima2018.xlsx
arimastlA2018.xlsx
arimastlB2018.xlsx
arimastlC2018.xlsx
ets2018.xlsx
etsstlA2018.xlsx
etsstlB2018.xlsx
etsstlC2018.xlsx
holt2018.xlsx
hw2018.xlsx
mint2018.xlsx
nnetar2018.xlsx
rwf2018.xlsx
snaive2018.xlsx
tbats2018.xlsx


C:\Users\rtz12\AppData\Local\Temp\ipykernel_15192\1896440418.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error=error.append(dic,ignore_index=True)


TDfa2018.xlsx
TDfe2018.xlsx
TDha2018.xlsx
TDhe2018.xlsx
wls2018.xlsx


C:\Users\rtz12\AppData\Local\Temp\ipykernel_15192\1896440418.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error=error.append(dic,ignore_index=True)


In [20]:
error1=error

In [30]:
error.mean(axis=0)

arima        1.217450
arimastlA    1.151192
arimastlB    1.280163
arimastlC    1.182958
ets          0.952258
hw           1.102756
mint         0.986145
nnetar       1.333729
snaive       1.464837
tbats        1.494949
TDfa         1.252022
TDfe         0.986026
wls          0.996086
TDha         2.167315
TDhe         2.180999
etsstlA      1.629210
etsstlB      1.754104
etsstlC      1.422899
holt         3.330711
rwf          1.874122
dtype: float64

In [27]:
np.diff(y_train.reshape(-1))

array([ 21459.,  -3207.,  13397.,  28060.,  22256.,   2269.,    160.,
       -15789.,   2079., -41746., -11818.,  -8435.,  18201.,  -6866.,
        16547.,  29238.,  10698.,   8241.,  -4493., -16771.,   4365.,
       -41911., -11294.])

In [ ]:
baseline_average_all=(baseline_average+baseline_weighted_average+baseline_stacking_ridge+baseline_stacking_lasso)/4
os.chdir(work_path)
baseline_average_all.to_excel('baseline.xlsx')

In [5]:
prediciton=pd.read_excel('reference.xlsx').iloc[:,1:]
real=pd.read_excel('real_reference.xlsx').iloc[:,1:]

In [26]:
model=pd.read_excel('reference_series'+'//'+'arimax_full_result.xlsx')
res=model.copy()
res[:]=0

for file in os.listdir('reference_series'):
    print(file)
    if file =='flight_result.xlsx':
        temp=pd.read_excel('reference_series'+'//'+file)
        res=res+temp
    # print(((prediciton-real.iloc[1:].reset_index()).abs().mean().iloc[1:]/real.diff().abs().mean()).mean())

arimax_full_result.xlsx
arimax_part_result.xlsx
ets_result.xlsx
flight_result.xlsx
holtwinters_result.xlsx
prophet_full_result.xlsx
prophet_part_result.xlsx
tbats_result.xlsx


In [27]:
temp

,Unnamed: 0,加拿大,智利,墨西哥,台湾,香港,日本,韩国,澳门,马尔代夫,...,印尼,新加坡,新西兰,美国,泰国,土耳其,澳大利亚,夏威夷,奥地利,捷克
0,2023-01-01,6946.804949,872.000000,10303.000000,9633.000000,280525.000000,31200.000000,24946.000000,9.916410e+05,5796.000000,...,27080.000000,28363.000000,4949.000000,63386.000000,91841.000000,8899.000000,14630.000000,1164.000000,4440.000000,2081.280521
1,2023-02-01,10067.486375,955.210730,9864.880057,10861.229192,48275.000000,36200.000000,21046.650019,1.238450e+06,32788.437802,...,48225.916379,35300.000000,8065.467956,40829.000000,255712.549339,10225.837869,84134.581235,1498.728307,2918.153530,4940.274181
2,2023-03-01,6628.418319,1070.815804,15490.807484,12847.555294,62930.471487,40800.694215,43769.917901,1.337879e+06,25756.441261,...,56721.474777,88545.438931,7423.390684,51774.725085,445128.604544,13385.733882,68415.919320,758.872337,7359.361708,6026.772524
3,2023-04-01,8431.674250,970.735458,8640.303777,15947.820511,80805.249615,53863.305469,112160.994535,1.465593e+06,36180.012315,...,69097.566723,112339.942248,9025.824947,55491.046941,660147.534396,23284.478405,72088.920010,1129.207918,13057.937680,8699.028682
4,2023-05-01,14794.700136,808.960846,9340.521608,17427.115526,86932.221583,57890.270525,155280.088837,1.488961e+06,55404.236439,...,79770.310711,113666.936781,7931.588169,76717.569175,700290.520271,31593.829739,58908.009848,1437.706215,16285.113061,11946.508084
5,2023-06-01,19162.872943,612.758592,8066.258476,16016.969460,84681.146397,71515.733031,173047.156866,1.375735e+06,61384.389590,...,92323.984598,128065.379121,4352.133516,90434.257606,664568.369491,30961.610786,66650.814052,1345.881552,17296.176122,13660.518986


In [25]:
real

,加拿大,智利,墨西哥,台湾,香港,日本,韩国,澳门,马尔代夫,柬埔寨,印尼,新加坡,新西兰,美国,泰国,土耳其,澳大利亚,夏威夷,奥地利,捷克
0,9217,1372,17627,5437,67244,33567,27367,336593,2468,16227,29642,19684,4731,48435,54146,10954,13880,2068,7166,9546
1,9244,872,10303,9633,280525,31200,24946,991641,5796,24978,27080,28363,4949,63386,91841,8899,14630,1164,4440,7757
2,8132,1034,9296,9346,1109885,36200,45884,1003999,4276,54457,36434,35300,4212,40829,155656,8888,40430,421,4514,6484
3,10053,1622,14175,10800,1970777,75700,73390,1242358,7619,53230,54549,60888,7119,67440,269773,13424,26810,1187,7328,11280
4,11072,1549,11569,14944,2308561,108295,105967,1386944,13504,51344,55090,90725,11237,118631,328414,17231,32000,635,12045,12910
5,16401,1251,11657,13601,2285859,134510,128171,1474304,15204,43738,64056,95613,8541,87622,285547,18308,32210,1510,13858,13252
6,21216,1326,11206,15117,2155143,208573,168035,1444933,17028,40383,62499,113290,8681,86392,311888,22215,37330,1136,19233,16406


In [23]:
((res/7-real.iloc[1:]).abs().mean()/real.diff().abs().mean()).reindex(real.columns).to_excel('error1.xlsx')

In [17]:
(res.iloc[:,1:]-real.iloc[1:]).abs().mean()/real.diff().abs().mean()

加拿大      0.880233
印尼       3.180832
台湾       1.370327
土耳其      3.328902
墨西哥      1.319732
夏威夷      0.483308
奥地利      1.217565
捷克       0.667924
新加坡      2.141256
新西兰      1.745077
日本       0.966973
智利       1.520923
柬埔寨      1.380486
泰国       5.571102
澳大利亚     4.832154
澳门       1.032843
美国       0.733403
韩国       1.146096
香港       3.804337
马尔代夫    11.257808
dtype: float64

In [62]:
(prediciton.iloc[-1]-real.iloc[-1])/real.iloc[-1]

加拿大    -0.241168
智利     -0.216325
墨西哥     0.040636
台湾     -0.323556
香港     -0.864724
日本     -0.805002
韩国     -0.559327
澳门     -0.519934
马尔代夫    0.499022
柬埔寨    -0.317739
印尼      0.607352
新加坡    -0.561537
新西兰    -0.603248
美国     -0.611879
泰国      0.198374
土耳其    -0.046475
澳大利亚   -0.294235
夏威夷     1.926829
奥地利    -0.271373
捷克     -0.052980
dtype: float64

加拿大       3530.833333
智利         189.166667
墨西哥       1429.000000
台湾        3510.333333
香港      438707.833333
日本       54671.333333
韩国       35629.166667
澳门      229553.833333
马尔代夫      4187.000000
柬埔寨       4680.166667
印尼        8646.333333
新加坡      35320.000000
新西兰       2912.000000
美国       32946.833333
泰国       65925.666667
土耳其       2859.333333
澳大利亚     14350.000000
夏威夷        452.000000
奥地利       3023.666667
捷克        2864.166667
dtype: float64